# NLP - Word Embeddings - Pascal Thürig

## Introduction
Starting point for this project is the following key requirements:
1. Use the BoolQ Dataset from Hugging Face
2. Use pre-trained model for word embeddings (word2vec, GloVe or fastText)
3. Train a 2-layer classifier with ReLU non-linearity

In this project I will be using pre-trained embeddings from word2vec and a simple 2-layer neural network to do the reading comprehension task on the BoolQ dataset.
I will document every decision made, from preprocessing to model training and evaluation. The goal is to classify each BoolQ question-answer pair as either 'Yes' or 'No'.

The data consists of the questions, a passage and the answer. In total there are 12'697 entries in the dataset. Splitting them into train (8427), validation (1000) and test (3270).

The following experiment have been performed:
- Learning rates: 1e-2 - 1e-6
- Batch Sizes: 64 - 512
- Epochs: 20 - 50
- Hidden Dimension: 64, 96, 128, 256
- Early stopping: 3 - 5




## TLDR; Here are the key decisions and justifications:
- BoolQ Dataset: Provided by Project berief
- Task: Classify BoolQ questions as either "yes" or "no" using pre-trained embeddings and a simple neural network
- Pre-trained embeddings: word2vec - Google News 300; for simplicity and already have a bit of experience with it
- Model: 2-layer NN with ReLU activation: Provided by Project brief
- Tokenizing: Yes, using a subword tokenizer.
- Lowercasing: Yes, all text will be lowercased.
- Stemming: No, stemming will not be applied.
- Lemmatizing: No, lemmatizing is not used initially but could be tried later.
- Stopword removal: No, stopwords are not removed to retain key information.
- Removal of other words: No, no other word removal is planned.
- Format cleaning: No further cleaning required, the dataset is already clean.
- Truncation: Yes, input text is truncated to a maximum of 512 tokens.
- Feature selection: None, relying on word2vec embeddings directly.
- Input format: Tokenized and padded sequences of word2vec embeddings.
- Label format: Binary (1 for "yes", 0 for "no").
- train/valid/test splits: 66% train, 8% validation, 26% test.
- Padding: Yes, sequences are padded for uniform input length.
- Embedding: Pre-trained word2vec embeddings are used for simplicity.
- Planned correctness tests: Shape consistency checks, binary label correctness, and validation of truncation and padding.
- Hyperparameters:
    - Learning Rate: 1e-2 – 1e-5
    - Batch Size: 16 - 64 (choosing maximum possible that my GPU can handle)
    - Epochs 10 - 50 in 5-/10-step increments
    - Hidden size: 64 - 512 
    - Early Stopping: Patience of 3 - 10 Epochs of non-improvement (depending on total epoch number)
- Evaluation: Accuracy and F1-Score: Accuracy for general performance and F1 to handle class imbalances
- Error Analysis: Investigating False Positives and False negatives to understand where the model fails

## Setup
Importing necessary libraries:
- datasets
- gensim
- transformers
- numpy
- torch
- wandb
- sklearn

In [ ]:
# Importing and installing all necessary libraries

%pip install datasets gensim transformers numpy torch wandb scikit-learn

from datasets import load_dataset
import gensim.downloader as api
import gensim
from transformers import AutoTokenizer
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import wandb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

First up the BoolQ dataset is loaded

In [ ]:
# Downloading the required BoolQ dataset and splitting it like required from the project presentation

train_data = load_dataset('google/boolq', split='train[:-1000]')
validation_data = load_dataset('google/boolq', split='train[-1000:]')
test_data = load_dataset('google/boolq', split='validation')


test_question = train_data[5]['question']
print(train_data[5])
print(f"Number of training samples: {len(train_data)}")
print(f"Number of validation samples: {len(validation_data)}")
print(f"Number of validation samples: {len(test_data)}")

For easy access during experiments I like to define the hyperparameters at the top of my notebooks

In retrospect defining my hyperparameters up here was more of a hassle than anything else I probably will not be doing this anymore, but you live and you learn...
As I did the experiments manually and not with a hyperparameter tuner like optuna I found myself constantly scrolling through the notebook.

In [ ]:
# Hyperparameters
EMBEDDING_DIM = 300
max_seq_length = 512

# Hyperparamters of my best performing run (val_loss)
run_number = 15
batch_size = 512
n_epochs = 50
learning_rate = 1e-5
hidden_dim = 256
patience = 5

# defining the WandB project and run name
wandb_project_name = "nlp-word_embeddings-pascal_thuerig"
wandb_run_name = f"run_{run_number}-batch_size_{batch_size}-n_epochs_{n_epochs}-lr_{learning_rate}-hidden_dim_{hidden_dim}"

# calculating the input and output dimension as I play around with the hidden dimension as a hyperparameter
sequence_length = max_seq_length * 2  # Concatenate question and passage
input_dim = sequence_length * EMBEDDING_DIM
output_dim = 2  # Binary classification

print(f"Seq Length: {sequence_length}")
print(f"Input dim: {input_dim}")

Now the pre-trained embeddings from word2vec - word2vec-google-news-300

This also changed: I am using fastText as I ran into lots of problems with subword embeddings and word2vec. Found out they seemingly don't match very well. I still wanted to keep the subword embeddings however, hoping for better performance. 

In [ ]:
model_name = "fasttext-wiki-news-subwords-300"
model_path = "fasttext-wiki-news-subwords-300.model"

# Check if the model file exists
try:
    # Load the model if it exists locally
    embeddings_model = gensim.models.KeyedVectors.load(model_path)
    print("Model loaded from local storage.")
except FileNotFoundError:
    # Download and save the model if it doesn't exist
    print("Downloading Word2Vec model...")
    embeddings_model = api.load(model_name)
    embeddings_model.save(model_path)  # Save the model locally
    print("Model downloaded and saved to local storage.")

## Preprocessing

The BoolQ data will be processed in the following way:
1.  Tokenizing: the input questions and passages using a subword tokenizer
2.  Lowercasing: the text for simplicity and to reduce the total vocabulary size
3.  Stemming: No, will not stem the words as to not lose information
4.  Lemmatizing: No, will try if it improves performance
5.  Stopword removal: No, will not be removed to not lose potentially critical information [research](https://datascience.stackexchange.com/questions/31048/pros-cons-of-stop-word-removal)
6.  Removal of other words: No, will not be removing any other words
7.  Format cleaning: The dataset is already sufficiently clean, it shouldn't impact performance
8.  Truncation: the input text is truncated to a maximum of 512 tokens
9.  Feature selection: Not applicable as we focus on raw text as input and leveraging the pre-trained word embeddings no further feature extraction is needed.
10. Input format: Will take the form of the tokenized and padded sequences of word embeddings
11. Label format: Binary labels "yes" or "no"
12. train/valid/test splits: Prerequisite to project (66/8/26)
13. Padding: the sequences is padded to ensure all inputs have the same length in each batch
14. Embedding: Using word2vec, solely for simplicity as I already know it.
15. Planned correctness tests: Check for shape mismatches between tokenized text and word embeddings. - Ensure that input sequences are properly truncated and padded. - Verify that binary labels are correctly assigned and match the expected outputs.


1. Preprocess text (lowercasing)

In [ ]:
# Lowercasing all text
def lowercase_text(text):
    return text.lower()

print(lowercase_text(test_question))

2. Tokenize with AutoTokenizer from Hugging Face
    I wanted to use the AutoTokenizer to get subword embeddings, hoping for a performance improvement. Also, I could easily pad and truncate the sequences within the tokenizer.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Found out I could pad and truncate straight in the tokenizer when reading the documentation
def tokenize_text(text):
    return tokenizer(text,
                     padding='max_length',    
                     truncation=True,         
                     max_length=max_seq_length,
                     return_tensors='pt')

tokenized_output = tokenize_text(test_question)

# Print tokens to check visually and manually if the look alright
print(tokenizer.convert_ids_to_tokens(tokenized_output['input_ids'][0]))
test_input_ids = tokenized_output['input_ids'][0]
print(test_input_ids)


3. Truncate or add padding -> *found out I can already do this in the tokenizer*

4. Preprocess Pipeline

Applying my little preprocessing that I am doing to the text

In [ ]:
def preprocess_pipeline(text):
    text = lowercase_text(text)
    tokens = tokenize_text(text)
    
    # Ensure tokenized length is correct
    assert tokens['input_ids'].shape[1] == max_seq_length, \
        f"Tokenized length is not equal to max_seq_length: {tokens['input_ids'].shape[1]}"
    
    token_strings = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    
    return token_strings

preprocessed_text = preprocess_pipeline(test_question)
print("Tokens:", preprocessed_text)


5. Embed tokens using word2vec (word2vec-google-news-300)

You might have guessed; Here I implement the fastText model and not the word2vec model anymore. The fastText model can handle oov words as well as the subword embeddings, this is why I switched and used it instead of word2vec.

In [ ]:
# I struggled with implementing this and getting an output that wasn't all zeros before switching to the fastText model. after changing out the model it started working reliably.
# This function is a mix of my own implementation according to documentation and some AI help until it worked.

def tokens_to_embeddings(tokens, embedding_model=embeddings_model, embedding_dim=EMBEDDING_DIM):
    embeddings = []
    for t in tokens:
        if t in embedding_model:
            embeddings.append(embedding_model[t])
        else:
            embeddings.append(np.zeros(embedding_dim))
    return np.array(embeddings)

embedded_text = tokens_to_embeddings(preprocessed_text)
print(embedded_text)

6. Create a custom BoolQ dataset class to:
    - get the data into a compatible format for the pyTorch dataloader.
    - organize question-answer pairs and apply the preprocessing pipeline.
    - easily batch, shuffle, and load the data during training.


Implemented this custom dataset class by following the documentation on [PyTorch](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files)

In [ ]:
class BoolQDataset(Dataset):
    def __init__(self, data, max_seq_length=max_seq_length):
        self.data = data
        self.max_seq_length = max_seq_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        question = self.data[idx]['question']
        passage = self.data[idx]['passage']
        
        label = 1 if self.data[idx]['answer'] else 0
        
        question_tokens = preprocess_pipeline(question)
        passage_tokens = preprocess_pipeline(passage)
        
        question_embeddings = tokens_to_embeddings(question_tokens)
        passage_embeddings = tokens_to_embeddings(passage_tokens)
        
        embeddings = np.concatenate((question_embeddings, passage_embeddings), axis=0).flatten()
        
        return torch.tensor(embeddings, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

7. Dataloaders as required by pyTorch

Implemented by following the [PyTorch documentation](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#preparing-your-data-for-training-with-dataloaders)

In [ ]:
train_dataset = BoolQDataset(train_data, embeddings_model)
validation_dataset = BoolQDataset(validation_data, embeddings_model)
test_dataset = BoolQDataset(test_data, embeddings_model)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

8. Initialize weights and biases for experiment tracking

In [ ]:
wandb.init(project=wandb_project_name, name=wandb_run_name, config={
    "learning_rate": learning_rate,
    "epochs": n_epochs,
    "batch_size": batch_size,
    "hidden_size": hidden_dim,
    "patience": patience,
})


## Model

The model architecture for this project is already fixed in the project brief as follows:
- **Network Architecture:** 2-Layer with ReLu non-linearity.
- **Loss / Optimizer:** Loss: CrossEntropyLoss / Optimizer: Adam (potentially trying SGD with or without momentum in experiments)
- **Experiments to run**: Mentioned in Training section below
- **Number of training runs**: Will depend on number of experiments
- **Checkpointing / Early stopping:** 3 - 10 epochs of non-improvement of the validation loss
- **Planned correctness tests:** Shape and Dimension consistency tests, Gradient Check, Sanity Check & Prediction Testd

1. Creating the neural network class:

In [ ]:
# Setting up the NN as required by the Project description as a 2-layer with ReLU non-linearity inbetween

class TwoLayerNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TwoLayerNN, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

2. Create instance of model and move it to the GPU

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TwoLayerNN(input_dim, hidden_dim, output_dim).to(device)

print(device)

3. Loss (nn.CrossEntropyLoss) and optimizer (optim.Adam)

CrossEntropy loss is used as it is the most commonly used loss for classification tasks
The Adam optimizer was chosen because it adjusts learning rates for faster, more efficient training, is particularly suited for complex, noisy NLP tasks.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

4. Training loop

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs, save_path="best_model.pth"):
    early_stop_counter = 0
    best_val_loss = np.inf
    best_model_wts = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        all_predictions = []
        all_labels = []
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs = val_inputs.to(device)
                val_labels = val_labels.to(device)
                val_outputs = model(val_inputs)
                
                # add to total validation loss
                val_loss += criterion(val_outputs, val_labels).item()
                
                # Convert outputs to predictions
                _, predicted = torch.max(val_outputs, 1)
                
                # Store predictions and true labels for metric calculation
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(val_labels.cpu().numpy())
        
        # Average validation loss
        avg_val_loss = val_loss / len(val_loader)
        avg_train_loss = running_loss / len(train_loader)
        
        # Calculate metrics with sklearn library
        accuracy = accuracy_score(all_labels, all_predictions)
        precision = precision_score(all_labels, all_predictions)
        recall = recall_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions)
        conf_matrix = confusion_matrix(all_labels, all_predictions)
        
        # print metrics and confusion matrix
        print(f"Epoch [{epoch+1}/{epochs}], Training loss: {avg_train_loss:.4f}, Validation loss: {avg_val_loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n {conf_matrix}")
        
        # logging to WandB
        wandb.log({
            "epoch": epoch + 1,
            "average_training_loss": avg_train_loss,
            "average_validation_loss": avg_val_loss,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "confusion_matrix": conf_matrix,
        })
        
        # Early stopping logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_wts = model.state_dict()
            torch.save(model.state_dict(), save_path)
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    print("Finished Training")
    return best_model_wts

## Training
Train the model with the following different hyperparameters:
- Learning rate: 1e-2 – 1e-5
- Batch size: 16 - 64
- Epochs: 10 - 50
- Hidden size: 64 - 512
- Early Stopping: Patience of 3 - 10 Epochs of non-improvement


In [ ]:
train_model(model, train_loader, validation_loader, criterion, optimizer, epochs=n_epochs)
wandb.finish()

## Evaluation
The model will be evaluated for the key metrics of:
- Accuracy
- Precision
- Recall
- F1 Score

The results will be averaged using micro averaging because I care about the total number of correct prediction regardless of the class ("yes" or "no"). 

Errors will be evaluated by making a confusion matrix and giving me the distribution of ture positives, false positives, true negatives and false negatives. Helping me figure out where the model is making most of it's mistakes.

Doing the evaluation in the training loop and logging everything to my [WandB](https://wandb.ai/aintnoair/nlp-word_embeddings-pascal_thuerig/overview)

The model is mostly predicting "yes" as the dataset is slightly imbalanced. 

## Experiments
### Batch size and learning rate

When testing different hyperparameter combinations it was clear quite quickly that a large batch size of 512 was performing consistently better than smaller batch sizes. So I stuck with that for further experiments and hyperparameter combinations. Changing the learning rate had inconsistent effects on performance. Lowering it all the way to 1e-2 would result in a drastic drop in the validation loss but would oscillate and very quickly get caught by the early stop criteria. Increasing the learning rate to 1e-6 however would result in extremely long and slowly improving runs with worse performance than a lower lr of 1e-5. I found 1e-5 to be the best performing learning rate.

### Hidden Size

Increasing the hidden dimension would drastically increase the runtime. Other than that it seemingly had next to no impact on the performance of the model. When increasing the hidden size the model would more consistently only predict a single class all the time as seen by the confusion matrix in [WandB](https://wandb.ai/aintnoair/nlp-word_embeddings-pascal_thuerig/overview)

### Epochs and early stopping

I found that early stopping after 5 epochs of non improvement was a good point to stop as it gave the model chance to recover. I have not experimented with the number of epochs as much as I had planned. I kept it at a maximum of 50 for most runs and let the early stopping criteria handle runs that were not improving anymore. This gave some runs the chance to take their time so to speak and gradually improve.

# Results

The runs delivered mixed and confusing results. For example the run with the best accuracy had one of the worst validation losses of all runs. The run with the best validation loss had one of the worst accuracies and would simply predict a single class over 80% of the time. The increase in batch size would result in the model predicting just one class most of the time when runs with smaller batch sizes were more even in their choice of class.

## Interpretation

To set concrete expectations for my model I take into account a couple of key benchmarks:
- **Accuracy:** Given the task of binary classification an accuracy of ~50% can be achieved with random guesses.
    - Expecting my model to hit an accuracy of ~60-75%.
- **F1 Score:** For this dataset I expect the F1 score to be similar to the accuracy of ~60-75%

Looking at the results in Weights and Biases it is difficult to say this model performs at all. My implementation could undoubtedly be improved upon however I believe a 2-layer model is just not able to depict the complexity that is required for a reading comprehension. I suspect this might even be the purpose of this first project to introduce us to the complexity that is nlp. 


## Stuff to improve:
- Experiment tracking
- Model evaluation
- Find out how an example project of this would look like

A big unknown for me in this project was the experiment tracking and evaluation of the model. I tried a lot that did not work at first and ran out of time in the end. This has been my first project I completed without a group, and it showed me how much I still don't know. Lacking experience and the knowledge of how to professionally complete such a project are areas I want to improve in during this semester. I want to learn how to not only get the model working but how to tune and optimize the hyperparameters, track everything and make it reproducible. These are all areas where I have little to no experience so far. I am looking forward to see how other students have tackled this project and try and learn from their implementation and get a better understanding of how to do such a project in general.